# Detectando Fumadores - Selección y Entrenamiento del Modelo

En este caso nos encontramos con un problema de clasificación binario ya que queremos detectar, según los biomarcadores, si los sujetos son fumadores o no para poder realizar intervenciones preventivas de promoción de la salud.

## Importando Bibliotecas

In [1]:
# Bibliotecas básicas de análisis de datos
import numpy as np
import pandas as pd

# Bibliotecas de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Herramientas de preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense


# Métricas de evaluación
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# Para balancear clases desbalanceadas
from imblearn.over_sampling import SMOTE

# Configuración de visualización
%matplotlib inline
sns.set_style("whitegrid")

print('Bibliotecas importadas correctamente')

2024-04-08 13:41:37.968265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Bibliotecas importadas correctamente


## Carga de Datos

Con el fin de preparar un único script para la ejecución del modelo, vamos a cargar los datos en bruto y con el conocimiento adquirido en el análisis realizado en el notebook anterior vamos a optimizar la limpieza y la adecuación de los datos en el siguiente apartado.

In [2]:
train = pd.read_csv('../data/raw/train.csv')
test =pd.read_csv('../data/raw/test.csv')
print('Archivos cargados')

Archivos cargados


## Limpieza y adecuación de datos

In [3]:
# Eliminamos la columna de id, ya que no aporta nada para el entrenamiento
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
print("ID's eliminados")

'''Creamos tres nuevas características con biomarcadores relacionados 
para ver si podemos reducir la dimensionalidad del conjunto de datos más adelante
relacionado con indices de obesidad que hemos visto en el EDA que afectan'''

def crear_caract(train):
    train['IMC'] = train['weight(kg)'] / ((train['height(cm)'] / 100) ** 2)#Indice de masa muscular
    train['HW_Ratio'] = train['height(cm)'] / train['waist(cm)']#Ratio entre altura y tamaño de la cintura
    train['HA_Ratio'] = train['height(cm)'] / train['age']#Ratio entre altura y edad
    return train

train = crear_caract(train)
test = crear_caract(test)

# Vemos el describe para asegurarnos que se ha realizado correctamente
descripcion = train.describe(include='all') # El include='all' para ver todas las columnas
descripcion.transpose() # Resulta más cómodo de visualizar al tener tantas columnas

ID's eliminados


,count,mean,std,min,25%,50%,75%,max
age,159256.0,44.306626,11.842286,20.000000,40.000000,40.000000,55.000000,85.000000
height(cm),159256.0,165.266929,8.818970,135.000000,160.000000,165.000000,170.000000,190.000000
weight(kg),159256.0,67.143662,12.586198,30.000000,60.000000,65.000000,75.000000,130.000000
waist(cm),159256.0,83.001990,8.957937,51.000000,77.000000,83.000000,89.000000,127.000000
eyesight(left),159256.0,1.005798,0.402113,0.100000,0.800000,1.000000,1.200000,9.900000
eyesight(right),159256.0,1.000989,0.392299,0.100000,0.800000,1.000000,1.200000,9.900000
hearing(left),159256.0,1.023974,0.152969,1.000000,1.000000,1.000000,1.000000,2.000000
hearing(right),159256.0,1.023421,0.151238,1.000000,1.000000,1.000000,1.000000,2.000000
systolic,159256.0,122.503648,12.729315,77.000000,114.000000,121.000000,130.000000,213.000000
relaxation,159256.0,76.874071,8.994642,44.000000,70.000000,78.000000,82.000000,133.000000


In [4]:
descripcion_test = test.describe(include='all') 
descripcion_test.transpose() 

,count,mean,std,min,25%,50%,75%,max
age,106171.0,44.426538,11.897138,20.000000,40.000000,40.000000,55.000000,85.000000
height(cm),106171.0,165.221322,8.837065,135.000000,160.000000,165.000000,170.000000,190.000000
weight(kg),106171.0,67.125618,12.586569,30.000000,60.000000,65.000000,75.000000,130.000000
waist(cm),106171.0,82.999892,8.946584,51.000000,77.000000,83.000000,89.000000,127.700000
eyesight(left),106171.0,1.004776,0.397690,0.100000,0.800000,1.000000,1.200000,9.900000
eyesight(right),106171.0,0.999483,0.385752,0.100000,0.800000,1.000000,1.200000,9.900000
hearing(left),106171.0,1.024216,0.153719,1.000000,1.000000,1.000000,1.000000,2.000000
hearing(right),106171.0,1.023980,0.152988,1.000000,1.000000,1.000000,1.000000,2.000000
systolic,106171.0,122.475403,12.765542,71.000000,114.000000,121.000000,130.000000,213.000000
relaxation,106171.0,76.820676,9.018782,40.000000,70.000000,78.000000,82.000000,140.000000


## Pruebas de Modelos

In [ ]:
#Preparamos los datos para las pruebas de modelos
X = train.drop('smoking', axis=1)
y = train['smoking']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Regresión Logística

In [6]:
# Entrena y evalúa Regresión Logística
log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

[[13508  4275]
 [ 3675 10394]]
              precision    recall  f1-score   support

           0       0.79      0.76      0.77     17783
           1       0.71      0.74      0.72     14069

    accuracy                           0.75     31852
   macro avg       0.75      0.75      0.75     31852
weighted avg       0.75      0.75      0.75     31852



### Árbol de Decisión 

In [7]:
# Entrena y evalúa Árbol de Decisión
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)  # No es necesario escalar para árboles de decisión
y_pred_tree = tree_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

[[13018  4765]
 [ 4996  9073]]
              precision    recall  f1-score   support

           0       0.72      0.73      0.73     17783
           1       0.66      0.64      0.65     14069

    accuracy                           0.69     31852
   macro avg       0.69      0.69      0.69     31852
weighted avg       0.69      0.69      0.69     31852



### Random Forest

In [8]:
# Entrena y evalúa Random Forest
forest_model = RandomForestClassifier()
forest_model.fit(X_train, y_train)
y_pred_forest = forest_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

[[13362  4421]
 [ 2827 11242]]
              precision    recall  f1-score   support

           0       0.83      0.75      0.79     17783
           1       0.72      0.80      0.76     14069

    accuracy                           0.77     31852
   macro avg       0.77      0.78      0.77     31852
weighted avg       0.78      0.77      0.77     31852



### KNN

In [9]:
# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print("Evaluando KNN:")
print(knn.score(X_test, y_test))

Evaluando KNN:
0.7053874168027126


### Naive Bayes

In [10]:
# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
print("Evaluando Naive Bayes:")
print(nb.score(X_test, y_test))

Evaluando Naive Bayes:
0.7197350244882582


### Support Vector Machine (Classifier)

In [11]:
# SVC
svc = SVC(kernel='linear')  # Puede que quieras cambiar el kernel y otros hiperparámetros
svc.fit(X_train, y_train)
print("Evaluando SVC:")
print(svc.score(X_test, y_test))

### Random Forest

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("Evaluando Random Forest:")
print(rf.score(X_test, y_test))

### Deep Learning

In [ ]:
# Deep Learning
# Definir la arquitectura de la red
model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Ajustar el modelo
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=1)

print("Evaluando Deep Learning:")
# Aquí se evaluará la precisión del modelo
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Suponiendo que ya tienes tus modelos entrenados: knn, nb, svc, rf, y model para Deep Learning

# KNN
y_pred_knn = knn.predict(X_test)
print("Matriz de Confusión para KNN:")
print(confusion_matrix(y_test, y_pred_knn))
print("Classification Report para KNN:")
print(classification_report(y_test, y_pred_knn))

# Naive Bayes
y_pred_nb = nb.predict(X_test)
print("Matriz de Confusión para Naive Bayes:")
print(confusion_matrix(y_test, y_pred_nb))
print("Classification Report para Naive Bayes:")
print(classification_report(y_test, y_pred_nb))

# SVC
y_pred_svc = svc.predict(X_test)
print("Matriz de Confusión para SVC:")
print(confusion_matrix(y_test, y_pred_svc))
print("Classification Report para SVC:")
print(classification_report(y_test, y_pred_svc))

# Random Forest
y_pred_rf = rf.predict(X_test)
print("Matriz de Confusión para Random Forest:")
print(confusion_matrix(y_test, y_pred_rf))
print("Classification Report para Random Forest:")
print(classification_report(y_test, y_pred_rf))

# Deep Learning
# Aquí deberás primero transformar las probabilidades en clases binarias, asumiendo una clase positiva si la probabilidad es mayor a 0.5
y_pred_dl = (model.predict(X_test) > 0.5).astype("int32")
print("Matriz de Confusión para Deep Learning:")
print(confusion_matrix(y_test, y_pred_dl))
print("Classification Report para Deep Learning:")
print(classification_report(y_test, y_pred_dl))
